## 1. Setup and Load Data
### Install Dependencies and Setup

In [1]:
%pip install tensorflow opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip list

Package                 Version
----------------------- -----------
absl-py                 2.2.2
asttokens               3.0.0
astunparse              1.6.3
certifi                 2025.1.31
charset-normalizer      3.4.1
comm                    0.2.2
contourpy               1.3.2
cycler                  0.12.1
debugpy                 1.8.14
decorator               5.2.1
executing               2.2.0
flatbuffers             25.2.10
fonttools               4.57.0
gast                    0.6.0
google-pasta            0.2.0
grpcio                  1.71.0
h5py                    3.13.0
idna                    3.10
ipykernel               6.29.5
ipython                 9.1.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
jupyter_client          8.6.3
jupyter_core            5.7.2
keras                   3.9.2
kiwisolver              1.4.8
libclang                18.1.1
Markdown                3.7
markdown-it-py          3.0.0
MarkupSafe              3.0.2
matplotlib           

In [9]:
# Import the required dependencies
import tensorflow as tf
import os

#### Clean the files and folders

In [41]:
#check and print the current Working Directory
wrk_dir = os.getcwd()
print(f"Current working directory is: {wrk_dir}")

Current working directory is: /home/dolaposalim/tensorflow-env/fila/filadentification


In [42]:
# Create a preferred working directory
wrk_dir_new = os.chdir("/home/dolaposalim/tensorflow-env/fila/filadentification")
print(f"New working directory is: {os.getcwd()}")

New working directory is: /home/dolaposalim/tensorflow-env/fila/filadentification


In [45]:
# to delete unwanted files in the directory
img_dir = os.path.join(wrk_dir, "img")
for image in os.listdir(img_dir):
    if image.lower().endswith(".jpg"):
        print(f"image: {image} is in the right format")
    elif image.lower().endswith(".jpeg"):
        print(f"image: {image} is in the right format")
    elif image.lower().endswith(".png"):
        print(f"image: {image} is in the right format")
    else:
        os.remove(os.path.join(img_dir, image))
        print(f"deleted non jpg image: {image}")

image: 149771532_MzAwLTQwMC01YWQ1NWM5YzY4.jpg is in the right format
image: images (100).jpeg is in the right format
image: images (180).jpeg is in the right format
image: image (21).jpeg is in the right format
image: images (215).jpeg is in the right format
image: images (114).jpeg is in the right format
image: image (1).jpeg is in the right format
image: images (123).jpeg is in the right format
image: images (254).jpeg is in the right format
image: images (104).jpeg is in the right format
image: images (146).jpeg is in the right format
image: images (88).jpeg is in the right format
image: images (273).jpeg is in the right format
image: images (187).jpeg is in the right format
image: images (198).jpeg is in the right format
image: asoebiguest-com-did-you-know-facts-about-the-fila-fila4_1.jpg is in the right format
image: images (51).jpeg is in the right format
image: images (122).jpeg is in the right format
image: images (274).jpeg is in the right format
image: images (42).jpeg is in 

In [46]:
img_folder = r"/home/dolaposalim/tensorflow-env/fila/filadentification/img"


# Get a sorted list
image_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_files.sort()


for index, filename in enumerate(image_files):
    ext = os.path.splitext(filename)[1]
    new_name = f"img_{index}{ext}"
    
    # Full old and new paths
    old_path = os.path.join(img_folder, filename)
    new_path = os.path.join(img_folder, new_name)
    
    # Rename
    os.rename(old_path, new_path)

print(" Renaming complete!")

 Renaming complete!


1